This notebook contains the code used to create the Entropy production Gaussian figure 3 in 2403.00679



In [ ]:
using DifferentialEquations;
using Plots;


In [ ]:
#parameters
T = 5
uref1 = 0.0
uref2 = 0.0
tau = 1
beta = 1;
L = 1;
m = 1;


epsilon = 0.2#1 #tau / sqrt(m * beta * (L^2) );
sf = epsilon^2;
g = 0.1 
omega = sqrt((1+g)/g)

samples = 500
t = LinRange(0,T,samples);


In [ ]:
#npvarsys

#this is the function for the pde we are solving
function evolution!(du, u, p, t)
    v11, v12, v22, c11, c12, c22 = u
    epsilon, uref2 = p
    du[1] = ((v12^2)/(2*g))-((g*(epsilon^2)*(-((c11*v11)^2)+((1+c12*v12)^2)))/(2*(c11^2))) - (((c12*v22/c11)^2)/(2*g)) - (epsilon/(c11^2))*(-(c11^2)*v11*v12 + c12*(1+c12*v12)*v22)
    du[2] = ((g*(epsilon^2)*v12*(1+c11*v11+c12*v12))/(2*c11))+((c12*(v22^2))/(2*g*c11))+(v12*(2*g+v22))/(2*g)+(epsilon/(2*c11))*(c11*((v12^2)+v11*(v22-2))+(1+2*c12*v12)*v22)
    du[3] = 2*(v22 - epsilon*v12 - 1)
    du[4] = -g*(epsilon^2)*(-1+c11*v11+c12*v12) - epsilon*(c11*v12+c12*(v22-2))
    du[5] = -(c11*v12)/(2*g)-(g*(epsilon^2)*c12*(1+c11*v11 + c12*v12))/(2*c11) - (c12*(2*g+v22))/(2*g) - (1/(2*c11))*epsilon*((c11^2)*v11 +c11*(1-2*c22+2*c12*v12)+(c12^2)*v22) 
    du[6] = 2 - 2*c22 - ((c12*v12)/g) - (((epsilon*c12)*(1+c11*v11 + c12*v12))/c11) - (v22*(c12^2))/(g*c11)
end

#the boundary condition function at the start
function bc1!(residual1, u, p, t)
    residual1[1] = u[1][4] - 1.0 #the solution at the start
    residual1[2] = u[1][5] - 0.0
    residual1[3] = u[1][6] - 1.0
    residual1[1] = u[end][4] - 2 #the solution at the end
    residual1[2] = u[end][5] - 0.0
    residual1[3] = u[end][6] - 1.0
end

tspan = (0.0,T)

p = [epsilon,uref2]

#initial guess
u0 = [0,0,0,1.0,0.0,1.0]

bvp1 = BVProblem(evolution!, bc1!, u0, tspan, p);
sol1 = solve(bvp1,MIRK6(), dt = 0.001)



In [ ]:
#functions
 
function v11(t) #
    return sol1(t)[1]
end

function v12(t) #
    return sol1(t)[2]
end

function v22(t) #
    return sol1(t)[3]
end

function c11(t) #position var
    return sol1(t)[4]
end

function c12(t) #cross var
    return sol1(t)[5]
end

function c22(t)  #mom var
    return sol1(t)[6]
end;


In [ ]:
plot(t,[v11.(t),v12.(t),v22.(t)])#,labels = ["Variance pos" "Cross-Variance" "Variance mom"])

In [ ]:
#matches!
plot(t,[c11.(t),c12.(t),c22.(t)],labels = ["Variance pos" "Cross-Variance" "Variance mom"])

In [ ]:
#meansys

#this is the function for  pde we are solving
function evolution!(du, u, p, t)
    
    s1,s2,v1,v2 = u
    epsilon,uref2 = p
    
    du[1] = (-1/2)*(g*(epsilon^2)*v1+g*(epsilon^2)*s1*v11(t)+epsilon*s1*v12(t)+epsilon*v2+epsilon*s2*(-2+g*epsilon*v12(t)+v22(t)))
  #-(g*(epsilon^2)/2)*(v1+s1*v11(t)+s2*v12(t)) -(epsilon/2)*(s1*v12(t) + v2+s2*(v22(t)-2))
       
    du[2] =(-1/2)*(1/g)*(epsilon*v1+g*epsilon*s1*v11(t)+s1*v12(t)+v2+s2*(2*g+g*epsilon*v12(t)+v22(t)))
  #(-1)*((epsilon/2)*(v1+s1*v11(t)+s2*v12(t)) + (1/(2*g))*(s1*v12(t)+v2+s2*(2*g+v22(t))))
    #   
    du[3] = (1/(2*(c11(t)^2)))*(1/g)*(s1*((g*epsilon+c12(t)*(g*epsilon*v12(t)+v22(t)))^2)+c11(t)*(-s2*(g*epsilon*v12(t)+v22(t))*(g*epsilon+c12(t)*(g*epsilon*v12(t)+v22(t)))+c11(t)*(g*epsilon*v1*(g*epsilon*v11(t)+v12(t))+g*epsilon*v11(t)*v2+v12(t)*v2)))
  #(1/(c11(t)^2))*(((g*(epsilon^2))/(2))*(s1*((1+c12(t)*v12(t))^2) + c11(t)*(c11(t)*v1*v11(t)-s1*v12(t)*(1+c12(t)*v12(t)))) - (epsilon/2)*(-(c11(t)^2)*(v1*v12(t)+v11(t)*v2)-2*c12(t)*s1*(1+c12(t)*v12(t))*v22(t)+c11(t)*s2*(1+2*c12(t)*v12(t))*v22(t)) - (1/(2*g))*(-(c12(t)^2)*s1*(v22(t)^2)+c11(t)*c12(t)*s2*(v22(t)^2)+(c11(t)^2)*(-v12(t)*v2)))
#      
    du[4] = (-1/(2*c11(t)))*(1/g)*(s1*(g*epsilon*v12(t)+v22(t))*(g*epsilon+c12(t)*(g*epsilon*v12(t)+v22(t)))-c11(t)*(2*g*v2+g*epsilon*v12(t)*v2+v2*v22(t)+g*epsilon*v1*(-2+g*epsilon*v12(t)+v22(t))+s2*((g*epsilon*v12(t)+v22(t))^2)))
 #(-1/(c11(t)))*((1/2)*g*(epsilon^2)*v12(t)*(s1*(1+c12(t)*v12(t))-c11(t)*(v1+s2*v12(t))) +(epsilon/2)*(s1*(1+2*c12(t)*v12(t))*v22(t)-c11(t)*(v1*(-2+v22(t))+v12(t)*(v2+2*s2*v22(t)))) +(1/(2*g))*(c12(t)*s1*(v22(t)^2) - c11(t)*(s2*(v22(t)^2)+v2*(2*g+v22(t)))))
#       
end

#the boundary condition function 
function bc!(residual, u, p, t)
    residual[1] = u[1][1] - 0.0 #the solution at the start
    residual[2] = u[1][2] - 0.0
    residual[3] = u[end][1] - sqrt(2) #the solution at the end
    residual[4] = u[end][2] - 0.0
end

tspan = (0.0,T)

#initial guess
u0 = [0.0, 0.0, 0.0, 0.0]

bvp2 = BVProblem(evolution!, bc!, u0, tspan, p);

#testing things out
meansyssol2 = solve(bvp2,MIRK6(), dt = 0.001)
#solve(bvp2,  MIRK4(), dt = 0.01, progress= true);




In [ ]:
#functions for easier naming
 
function s1(t) #position mean
    return meansyssol2(t)[1]
end

function s2(t) #momentummean
    return meansyssol2(t)[2]
end

function v1(t)
    return meansyssol2(t)[3]
end

function v2(t)
    return meansyssol2(t)[4]
end;


In [ ]:

plot(t,[v1.(t),v2.(t)])#,v1.(t),v2.(t)])

In [ ]:
plot(t,[s1.(t),s2.(t)])#,v1.(t),v2.(t)])

In [ ]:
##perturbative expansion  

p = [T,uref1, uref2];

#this is the function for the pde we are solving
function evolution!(du, u, p, t)
    nu_t, mu_t, val_1, val_2  = u
    T, uref1, uref2 = p
    
    coeff_mu = (1/8)*(1/sinh(T*omega/2))*((1/cosh(T*omega/2))^2)*(cosh(2*T)+sinh(2*T))*(8*omega*(cosh(T)^2)*cosh(T*omega)+8*omega*(-1+cosh(T*omega)*(sinh(T)^2))-2*((omega^2)+4)*sinh(2*T)*sinh(T*omega))
    coeff_a1 = (1/4)*(1/sinh(T*omega/2))*((1/cosh(T*omega/2))^2)*(cosh(2*T)+sinh(2*T))*(8*omega*(cosh(T)^2)*cosh(T*omega)+8*omega*(-1+cosh(T*omega)*(sinh(T)^2))-2*((omega^2)+4)*sinh(2*T)*sinh(T*omega))
    
    #positin var
    du[1] = -sf*((omega*(-2*T*omega*cosh((T*omega)/2)*sinh(T)+(T*(omega^2)*cosh(T)-((omega^2)-4)*sinh(T))*sinh(T*omega/2))*nu_t*val_2)/(T*((omega^2)-1)*(-2*cosh(T*omega/2)*sinh(T)+omega*cosh(T)*sinh(T*omega/2))))
    
    du[2] = sf*((-1/(coeff_mu*(-2)*nu_t*2*T*((omega^2)-1)*(-2*(1+cosh(T*omega))*sinh(T)+omega*cosh(T)*sinh(T*omega))))*
    (coeff_mu*(-2*nu_t*(omega*(-2*sinh(T)*(T*omega+T*omega*cosh(T*omega)-2*sinh(T*omega))+omega*cosh(T)*(2-2*cosh(T*omega)+T*omega*sinh(T*omega)))*val_1 + omega*(-omega*cosh(T)*(-2+2*cosh(T*omega)+T*omega*sinh(T*omega))+2*sinh(T)*(T*omega+T*omega*cosh(T*omega)+((omega^2)-2)*sinh(T*omega)))*mu_t*val_2) + 4*T*(-1+(omega^2))*(-2*(1+cosh(T*omega))*sinh(T)+omega*cosh(T)*sinh(T*omega))*mu_t*(du[1]/sf))))
    #(1/(4*T*nu_t*((omega^2)-1)*(-2*(cosh(T*omega)+1)*sinh(T)+omega*cosh(T)*sinh(T*omega))))*(-2*nu_t*(omega*(-2*sinh(T)*(T*omega+T*omega*cosh(T*omega)-2*sinh(T*omega))+omega*cosh(T)*(2-2*cosh(T*omega)+T*omega*sinh(T*omega)))*val_1)+omega*(-omega*cosh(T)*(-2+2*cosh(T*omega)+T*omega*sinh(T*omega))+2*sinh(T)*(T*omega+T*omega*cosh(T*omega)+((omega^2)-2)*sinh(T*omega)))*mu_t*val_2 + 4*T*((omega^2)-1)*(-2*(1+cosh(T*omega))*sinh(T)+omega*cosh(T)*sinh(T*omega))*mu_t*du[1])
    
    du[3] = (sf)*((-1/(coeff_a1*2*T*((omega^2)-1)*(2*(1+cosh(T*omega))*sinh(T)-omega*cosh(T)*sinh(T*omega))))*(coeff_a1*(2*omega*(omega/tanh(T)-2/tanh(T*omega/2))*(-2+T*omega/tanh(T*omega/2))*sinh(T)*(sinh(T*omega/2)^2)*val_1*val_2 + (omega^2)*(-2*cosh(T)*(-1+cosh(T*omega))+omega*sinh(T)*sinh(T*omega))*mu_t*(val_2^2))))
    #(2*omega*(omega/tanh(T) - 2/tanh(T*omega/2))*(-2+T*omega/tanh(T*omega/2))*sinh(T)*(sinh(T*omega/2)^2)*val_1*val_2 + (omega^2)*(-2*cosh(T)*(-1+cosh(T*omega))+omega*sinh(T)*sinh(T*omega))*mu_t*(val_2^2))/(-(2*T*((omega^2)-1)*(2*(1+cosh(T*omega))*sinh(T)-omega*cosh(T)*sinh(T*omega))))
    
    du[4] = sf*(-(omega*(val_2^2)*(2*T*omega*cosh(T*omega/2)*sinh(T)+(-T*(omega^2)*cosh(T)+((omega^2)-4)*sinh(T))*sinh(T*omega/2)))/(2*T*((omega^2)-1)*(-2*cosh(T*omega/2)*sinh(T)+omega*cosh(T)*sinh(T*omega/2))))

     
   
end

#the boundary condition function 
function bc!(residual, u, p, t)
    residual[1] = u[1][1] - 1.0 #the solution at the start
    residual[2] = u[1][2] - 0.0 #the solution at the start
    residual[3] = u[end][1] - 1.7 #the solution at the end
    residual[4] = u[end][2] - sqrt(2) #the solution at the start
end


tspan = (0.0,T)

#initial guess
u0 = [1.0,0.0,0.0,0.0]

bvp1 = BVProblem(evolution!, bc!, u0, tspan, p);

perturb_sol1 = solve(bvp1,Shooting(KenCarp4()), dt = 0.0001);
    #MIRK4(), dt = 0.01);
    ##



In [ ]:
function nu(t) #positin var
    return perturb_sol1(t)[1]
end

function mu(t) 
    return perturb_sol1(t)[2]
end

function val_1(t) #value
    return perturb_sol1(t)[3]
end

function val_2(t) #value
    return perturb_sol1(t)[4]
end

#meanmom[1]
function mom_mean(t) #f^11_0,t_2
      return epsilon*(1/cosh((T*omega)/2)*(sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)+(omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2)))*mu(t)*val_2(t)+(1/cosh((T*omega)/2)*(2*sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)*(2-2*exp(2*T)+(1+exp(2*T))*omega*tanh((T*omega)/2))*val_1(t)-omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega))*mu(t)*val_2(t)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2)))
end;


In [ ]:
#matches!
plot(t,[nu.(t),mu.(t),val_1.(t),val_2.(t)],layout = 4)


In [ ]:
#updated


#varmom1
function mom_var(t)
    return 1/2*sech((T*omega)/2)^2*(sinh((t*omega)/2)*sinh(1/2*(t - T)*omega)+(omega*sech((T*omega)/
        2)*(-exp(2*T)*sinh(t*omega)+sinh((t - T)*omega) + exp(2*t)*sinh(T*omega)))/
        (4 - 4*exp(2*T) + 2*(1 + exp(2*T))*omega*tanh((T*omega)/2)))^2*(mu(t)^2 + nu(t))*val_2(t)^2+(sech((T*omega)/2)^2*(omega*sech((T*omega)/
        2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega))+2*exp(2*t)*omega*sinh((T*omega)/2) + 
      2*sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)*(2-2*exp(2*T)+(1+exp(2*T))*omega*tanh((T*omega)/
           2)))*mu(t)*val_2(t)*(2*sinh((t*omega)/2)*sinh(1/2*(t - T)*omega)*(2 - 
         2*exp(2*T)+(1+exp(2*T))*omega*tanh((T*omega)/2))*val_1(t) - omega*sech((T*omega)/
        2)*(-exp(2*T)*sinh(t*omega) + sinh((t - T)*omega) + 
         exp(2*t)*sinh(T*omega))*mu(t)*val_2(t)))/(4*(2-2*exp(2*T)+(1+exp(2*T))*omega*tanh((T*omega)/2))^2) + 
 1/8*sech((T*omega)/2)*(4*sech((T*omega)/2)*(sinh((t*omega)/2)*sinh(1/2*(t - T)*omega) + (omega*sech((T*omega)/
           2)*(-exp(2*T)*sinh(t*omega) + sinh((t - T)*omega) + 
            exp(2*t)*sinh(T*omega)))/(4 - 4*exp(2*T) + 2*(1 + exp(2*T))*omega*tanh((T*omega)/2)))^2*val_1(t)^2 + 1/(-4 + omega^2)*csch((T*omega)/
       2)*(-2*omega*cosh(t*omega) + 2*omega*cosh((t - T)*omega) + 4*sinh(t*omega) - 
        4*sinh((t - T)*omega) - 4*sinh(T*omega) + omega^2*sinh(T*omega) - 
        exp(-2*t)*omega*(-2 + 2*cosh(T*omega) + omega*sinh(T*omega)) - (omega*sinh((T*omega)/
            2)*(-4*exp(2*T)*omega*cosh(t*omega) + 4*omega*cosh((t - T)*omega) + 
             8*exp(2*T)*sinh(t*omega) - 8*sinh((t - T)*omega) - 4*exp(2*t)*sinh(T*omega) + 
             exp(2*t)*omega^2*sinh(T*omega)))/(-2*(-1 + exp(2*T))*cosh((T*omega)/2) + 
        (1 + exp(2*T))*omega*sinh((T*omega)/2)) + (exp(-2*t)*omega*sinh((T*omega)/2)*(-4*exp(2*T)*omega + 
         4*omega*cosh(T*omega) + (4 + omega^2)*sinh(T*omega)))/(-2*(-1 + exp(2*T))*cosh((T*omega)/
        2) + (1 + exp(2*T))*omega*sinh((T*omega)/2)))*val_2(t) - (8*omega*sech((T*omega)/2)^2*(-exp(2*T)*sinh(t*omega) + sinh((t - T)*omega) + exp(2*t)*sinh(T*omega))*(sinh((t*omega)/2)*sinh(
           1/2*(t - T)*omega) + (omega*sech((T*omega)/2)*(-exp(2*T)*sinh(t*omega) + sinh((t - T)*omega) + exp(2*t)*sinh(T*omega)))/(4 - 4*exp(2*T) + 
            2*(1 + exp(2*T))*omega*tanh((T*omega)/2)))*val_1(t)*(val_1(t) + mu(t)*val_2(t)))/(4 - 4*exp(2*T) + 2*(1 + exp(2*T))*omega*tanh((T*omega)/
         2)) + (omega^2*sech((T*omega)/2)^3*(-exp(2*T)*sinh(t*omega) + sinh((t - T)*omega) + exp(2*t)*sinh(T*omega))^2*(val_1(t) + mu(t)*val_2(t))^2)/(2 - 
       2*exp(2*T) + (1 + exp(2*T))*omega*tanh((T*omega)/2))^2)
      
    #1/2/cosh((T*omega)/2)^2*(sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)+(omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2)))^2*(mu(t)^2+nu(t))*val_2(t)^2+(1/cosh((T*omega)/2)^2*(omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega))+2*exp(2*t)*omega*sinh((T*omega)/2)+2*sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)*(2-2*exp(2*T)+(1+exp(2*T))*omega*tanh((T*omega)/2)))*mu(t)*val_2(t)*(2*sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)*(2-2*exp(2*T)+(1+exp(2*T))*omega*tanh((T*omega)/2))*val_1(t)-omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega))*mu(t)*val_2(t)))/(4*(2-2*exp(2*T)+(1+exp(2*T))*omega*tanh((T*omega)/2))^2)+1/8*(4/cosh((T*omega)/2)^2*(sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)+(omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2)))^2*val_1(t)^2-(2*exp(-2*t)*(-2*(1+exp(2*T))+(-1+exp(2*T))*omega/tanh((T*omega)/2))*((-1+exp(2*t))*(exp(2*t)-exp(2*T))*omega+2*exp(2*t)*(-1+exp(2*T))/tanh((T*omega)/2)-2*exp(2*t)*(-1+exp(2*T))*cosh(t*omega-(T*omega)/2)/sinh((T*omega)/2))*tanh((T*omega)/2)*val_2(t))/((-1+exp(4*T))*(4+omega^2)-4*(1+exp(4*T))*omega/tanh(T*omega)+8*exp(2*T)*omega/sinh(T*omega))-(8*omega/cosh((T*omega)/2)^3*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega))*(sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)+(omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2)))*val_1(t)*(val_1(t)+mu(t)*val_2(t)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2))+(omega^2/cosh((T*omega)/2)^4*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega))^2*(val_1(t)+mu(t)*val_2(t))^2)/(2-2*exp(2*T)+(1+exp(2*T))*omega*tanh((T*omega)/2))^2)

end

plot(t,mom_var.(t))

In [ ]:
##check this!!!

function cross_cor(t)
    return 1/cosh((T*omega)/2)*(sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)+(omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2)))*(mu(t)^2+nu(t))*val_2(t)+(1/cosh((T*omega)/2)*mu(t)*(2*sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)*(2-2*exp(2*T)+(1+exp(2*T))*omega*tanh((T*omega)/2))*val_1(t)-omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega))*mu(t)*val_2(t)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2))
end

plot(cross_cor.(t))

In [ ]:
##MEANPOSCOR1

function mean_pos_cor(t)
    return (epsilon^2)*((1/(2*(-1+omega^2)))*mu(t)*((2/cosh((T*omega)/2)^2*(2*exp(T)*sinh(T)*(-2+2*cosh(t*omega)+2*cosh((t-T)*omega)-2*cosh(T*omega)+omega*(sinh(t*omega)+sinh((t-T)*omega)))+(1-2*exp(2*t)+exp(2*T))*omega*sinh(T*omega)))/((8-8*exp(2*T)+4*(1+exp(2*T))*omega*tanh((T*omega)/2))*nu(t))+(1/nu(t))*2/cosh((T*omega)/2)*(sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)+(omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2)))+(omega/cosh((T*omega)/2)*sinh(t*omega-(T*omega)/2)+((-2*t+T)*omega*tanh((T*omega)/2))/T+(omega^2*sinh((T*omega)/2)*(-T*omega+exp(2*t)*T*omega-2*exp(2*T)*T*(-1+cosh(t*omega))/sinh(T*omega)-2*exp(T)*t*omega*sinh(T)+4*T/tanh(T*omega)*sinh((t*omega)/2)^2-2*T*sinh(t*omega)+4*exp(T)*t*cosh(T)*tanh((T*omega)/2)))/(T*(-2*(-1+exp(2*T))*cosh((T*omega)/2)+(1+exp(2*T))*omega*sinh((T*omega)/2)))+2/cosh((T*omega)/2)*(sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)+(omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2))))*val_2(t))-(1/(2*(-1+omega^2)))*((2/cosh((T*omega)/2)^2*(2*exp(T)*sinh(T)*(-2+2*cosh(t*omega)+2*cosh((t-T)*omega)-2*cosh(T*omega)+omega*(sinh(t*omega)+sinh((t-T)*omega)))+(1-2*exp(2*t)+exp(2*T))*omega*sinh(T*omega))*mu(t))/((8-8*exp(2*T)+4*(1+exp(2*T))*omega*tanh((T*omega)/2))*nu(t))+(1/nu(t))*2/cosh((T*omega)/2)*(sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)+(omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2)))*mu(t)-(omega/cosh((T*omega)/2)*sinh(t*omega-(T*omega)/2)+((-2*t+T)*omega*tanh((T*omega)/2))/T+(omega^2*sinh((T*omega)/2)*(-T*omega+exp(2*t)*T*omega-2*exp(2*T)*T*(-1+cosh(t*omega))/sinh(T*omega)-2*exp(T)*t*omega*sinh(T)+4*T/tanh(T*omega)*sinh((t*omega)/2)^2-2*T*sinh(t*omega)+4*exp(T)*t*cosh(T)*tanh((T*omega)/2)))/(T*(-2*(-1+exp(2*T))*cosh((T*omega)/2)+(1+exp(2*T))*omega*sinh((T*omega)/2)))+2/cosh((T*omega)/2)*(sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)+(omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2))))*val_1(t)+(omega/cosh((T*omega)/2)^2*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega))*(val_1(t)+mu(t)*val_2(t)))/(2-2*exp(2*T)+(1+exp(2*T))*omega*tanh((T*omega)/2))+(omega^2*sinh((T*omega)/2)*(-T*omega+exp(2*t)*T*omega-2*exp(2*T)*T*(-1+cosh(t*omega))/sinh(T*omega)-2*exp(T)*t*omega*sinh(T)+4*T/tanh(T*omega)*sinh((t*omega)/2)^2-2*T*sinh(t*omega)+4*exp(T)*t*cosh(T)*tanh((T*omega)/2))*(val_1(t)+mu(t)*val_2(t)))/(T*(-2*(-1+exp(2*T))*cosh((T*omega)/2)+(1+exp(2*T))*omega*sinh((T*omega)/2)))))
    
end

plot(mean_pos_cor.(t))

In [ ]:
#updated

function var_pos_cor(t)
    return 2*sf*(1/(2*(-1 + omega^2))*(mu(t)^2 + nu(t))*((2*sech((T*omega)/2)^2*(2*exp(T)*sinh(T)*
                    (-2 + 2*cosh(t*omega) + 2*cosh((t - T)*omega) - 2*cosh(T*omega) + omega*(sinh(t*omega) + 
                  sinh((t - T)*omega))) + (1 - 2*exp(2*t) + exp(2*T))*omega*sinh(T*omega)))/((8 - 8*exp(2*T) + 
            4*(1 + exp(2*T))*omega*tanh((T*omega)/2))*nu(t)) + (2*sech((T*omega)/2)*(sinh((t*omega)/2)*
            sinh(1/2*(t - T)*omega) + (omega*sech((T*omega)/2)*(-exp(2*T)*sinh(t*omega) + sinh((t - T)*omega) + 
             exp(2*t)*sinh(T*omega)))/( 4 - 4*exp(2*T) + 2*(1 + exp(2*T))*omega*tanh((T*omega)/2))))/nu(t)+ (omega*sech((T*omega)/2)*
                sinh(t*omega - (T*omega)/2) + ((-2*t + T)*omega*tanh((T*omega)/2))/T + 
                (omega^2*sinh((T*omega)/2)*(-T*omega + exp(2*t)*T*omega - 
               2*exp(2*T)*T*(-1 + cosh(t*omega))*csch(T*omega) - 
               2*exp(T)*t*omega*sinh(T) + 4*T*coth(T*omega)*sinh((t*omega)/2)^2 - 
               2*T*sinh(t*omega) + 4*exp(T)*t*cosh(T)*tanh((T*omega)/2)))/
            (T*(-2*(-1 + exp(2*T))*cosh((T*omega)/2) + (1 + exp(2*T))*omega*sinh((T*omega)/2))) + 
          2*sech((T*omega)/2)*(sinh((t*omega)/2)*sinh(1/2*(t - T)*omega) + (omega*sech((T*omega)/
               2)*(-exp(2*T)*sinh(t*omega) + 
                sinh((t - T)*omega) + exp(2*t)*sinh(T*omega)))/(
             4 - 4*exp(2*T) + 2*(1 + exp(2*T))*omega*tanh((T*omega)/2))))*val_2(t)) - 
        1/(2*(-1 + omega^2))*mu(t)*((2*sech((T*omega)/2)^2*(2*exp(T)*sinh(T)*(-2 + 2*cosh(t*omega) + 
               2*cosh((t - T)*omega) - 2*cosh(T*omega) + omega*(sinh(t*omega) + sinh((t - T)*omega))) + 
                (1 - 2*exp(2*t) + exp(2*T))*omega*sinh(T*omega))*mu(t))/((8 - 8*exp(2*T) + 
            4*(1 + exp(2*T))*omega*tanh((T*omega)/2))*nu(t)) + (2*sech((T*omega)/2)*(sinh((t*omega)/2)*
                sinh(1/2*(t - T)*omega) + (omega*sech((T*omega)/
            2)*(-exp(2*T)*sinh(t*omega) + sinh((t - T)*omega) + 
            exp(2*t)*sinh(T*omega)))/(4 - 4*exp(2*T) + 
           2*(1 + exp(2*T))*omega*tanh((T*omega)/2)))*mu(t))/nu(t) 
        - (omega*sech((T*omega)/2)*sinh(t*omega - (T*omega)/2) + ((-2*t + T)*omega*tanh((T*omega)/2))/
          T + (omega^2*sinh((T*omega)/2)*(-T*omega +exp(2*t)*T*omega - 
               2*exp(2*T)*T*(-1 + cosh(t*omega))*csch(T*omega) - 
               2*exp(T)*t*omega*sinh(T) + 4*T*coth(T*omega)*sinh((t*omega)/2)^2 - 
               2*T*sinh(t*omega) + 4*exp(T)*t*cosh(T)*tanh((T*omega)/
                 2)))/(T*(-2*(-1 + exp(2*T))*cosh((T*omega)/
                 2) + (1 + exp(2*T))*omega*sinh((T*omega)/2))) + 2*sech((T*omega)/2)*(sinh((t*omega)/2)*
                sinh(1/2*(t - T)*omega) + (omega*sech((T*omega)/2)*(-exp(2*T)*sinh(t*omega) + 
                sinh((t - T)*omega) + exp(2*t)*sinh(T*omega)))/
                (4 - 4*exp(2*T) + 2*(1 + exp(2*T))*omega*tanh((T*omega)/2))))*val_1(t) + 
            (omega*sech((T*omega)/2)^2*(-exp(2*T)*sinh(t*omega) + sinh((t - T)*omega) + 
         exp(2*t)*sinh(T*omega))*(val_1(t) + mu(t)*val_2(t)))/(
       2 - 2*exp(2*T) + (1 + exp(2*T))*omega*tanh((T*omega)/2)) 
            + (omega^2*sinh((T*omega)/2)*(-T*omega +exp(2*t)*T*omega 
            - 2*exp(2*T)*T*(-1 + cosh(t*omega))*csch(T*omega) - 2*exp(T)*t*omega*sinh(T) + 4*T*coth(T*omega)*sinh((t*omega)/2)^2 
            - 2*T*sinh(t*omega) +4*exp(T)*t*cosh(T)*tanh((T*omega)/2))*(val_1(t) + mu(t)*val_2(t)))/(T*(-2*(-1 + exp(2*T))*cosh((T*omega)/2) + (1 + exp(2*T))*omega*sinh((T*omega)/2)))))
    
    #2*epsilon^2*(1/(2*(-1+omega^2))*(mu(t)^2+nu(t))*((2/cosh((T*omega)/2)^2*(2*exp(T)*sinh(T)*(-2+2*cosh(t*omega)+2*cosh((t-T)*omega)-2*cosh(T*omega)+omega*(sinh(t*omega)+sinh((t-T)*omega)))+(1-2*exp(2*t)+exp(2*T))*omega*sinh(T*omega)))/((8-8*exp(2*T)+4*(1+exp(2*T))*omega*tanh((T*omega)/2))*nu(t))+(1/nu(t))*2/cosh((T*omega)/2)*(sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)+(omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)*+exp(2*t)*sinh(T*omega)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2)))+(omega/cosh((T*omega)/2)*sinh(t*omega-(T*omega)/2)+((-2*t+T)*omega*tanh((T*omega)/2))/T+(omega^2*sinh((T*omega)/2)*(-T*omega+exp(2*t)*T*omega-2*exp(2*T)*T*(-1+cosh(t*omega))/sinh(T*omega)-2*exp(T)*t*omega*sinh(T)+4*T/tanh(T*omega)*sinh((t*omega)/2)^2-2*T*sinh(t*omega)+4*exp(T)*t*cosh(T)*tanh((T*omega)/2)))/(T*(-2*(-1+exp(2*T))*cosh((T*omega)/2)+(1+exp(2*T))*omega*sinh((T*omega)/2)))+2/cosh((T*omega)/2)*(sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)+(omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2))))*val_2(t))-1/(2*(-1+omega^2))*mu(t)*((2/cosh((T*omega)/2)^2*(2*exp(T)*sinh(T)*(-2+2*cosh(t*omega)+2*cosh((t-T)*omega)-2*cosh(T*omega)+omega*(sinh(t*omega)+sinh((t-T)*omega)))+(1-2*exp(2*t)+exp(2*T))*omega*sinh(T*omega))*mu(t))/((8-8*exp(2*T)+4*(1+exp(2*T))*omega*tanh((T*omega)/2))*nu(t))+(1/nu(t))*2/cosh((T*omega)/2)*(sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)+(omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)*+exp(2*t)*sinh(T*omega)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2)))*mu(t)-(omega/cosh((T*omega)/2)*sinh(t*omega-(T*omega)/2)+((-2*t+T)*omega*tanh((T*omega)/2))/T+(omega^2*sinh((T*omega)/2)*(-T*omega+exp(2*t)*T*omega-2*exp(2*T)*T*(-1+cosh(t*omega))/sinh(T*omega)-2*exp(T)*t*omega*sinh(T)+4*T/tanh(T*omega)*sinh((t*omega)/2)^2-2*T*sinh(t*omega)+4*exp(T)*t*cosh(T)*tanh((T*omega)/2)))/(T*(-2*(-1+exp(2*T))*cosh((T*omega)/2)+(1+exp(2*T))*omega*sinh((T*omega)/2)))+2/cosh((T*omega)/2)*(sinh((t*omega)/2)*sinh(1/2*(t-T)*omega)+(omega/cosh((T*omega)/2)*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega)))/(4-4*exp(2*T)+2*(1+exp(2*T))*omega*tanh((T*omega)/2))))*val_1(t)+(omega/cosh((T*omega)/2)^2*(-exp(2*T)*sinh(t*omega)+sinh((t-T)*omega)+exp(2*t)*sinh(T*omega))*(val_1(t)+mu(t)*val_2(t)))/(2-2*exp(2*T)+(1+exp(2*T))*omega*tanh((T*omega)/2))+(omega^2*sinh((T*omega)/2)*(-T*omega+exp(2*t)*T*omega-2*exp(2*T)*T*(-1+cosh(t*omega))/sinh(T*omega)-2*exp(T)*t*omega*sinh(T)+4*T/tanh(T*omega)*sinh((t*omega)/2)^2-2*T*sinh(t*omega)+4*exp(T)*t*cosh(T)*tanh((T*omega)/2))*(val_1(t)+mu(t)*val_2(t)))/(T*(-2*(-1+exp(2*T))*cosh((T*omega)/2)+(1+exp(2*T))*omega*sinh((T*omega)/2)))))
    
end

plot(var_pos_cor.(t))

In [ ]:
###MATCHES!

#position mean
p1 = plot(t, [mu.(t).+ mean_pos_cor.(t),s1.(t)],
    labels = ["eps = $epsilon" "non-perturbative"],
    title ="Position mean",
    titlefontsize = 12,
    legend =false)

plot(p1)

In [ ]:
#momentum mean
p2 = plot(t, [mom_mean.(t),s2.(t)],
    labels = ["eps =$epsilon" "non-perturbative"],
    titlefontsize = 12,
    title = "momentum mean",
    legend =true)


plot(p2)

In [ ]:
#matches

function pos_var_plot(t)
    return nu(t) + var_pos_cor(t) - 2*mu(t)*mean_pos_cor(t)
end

p3 = plot(t, [pos_var_plot.(t),c11.(t)],
    labels = ["eps =$epsilon" "non-perturbative"],
    titlefontsize = 12,
    title = "Position Variance",
    legend =true)

plot(p3)

In [ ]:
#CROSS VAR

function cross_cor_plot(t)
    return epsilon*cross_cor(t)-mom_mean(t)*mu(t)
end

p4 = plot(t, [cross_cor_plot.(t),c12.(t)],
    labels = ["eps =$epsilon" "non-perturbative"],
    titlefontsize = 12,
    title = "Cross-variance",
    legend =true)



In [ ]:
#MOM VAR


function mom_var_plot(t)
    
    return 1 + 2*sf*mom_var(t) - ((mom_mean(t))^2)
end

p5 = plot(t, [mom_var_plot.(t),c22.(t)],
    labels = ["eps = $epsilon" "non-perturbative"],
    titlefontsize = 12,
    title="momentum variance",
    legend =true)

plot(p5)

In [ ]:
###MATCHES!
p6 = plot([zeros(1),zeros(2)],alpha=[1,1],labels = ["eps = $epsilon" "non-perturbative"],axis=([], false),leg_position = "left")

#position mean
plot(p1,p2,p3,p4,p5, p6,legend = [false false false false false true],layout = 6)

#T=2

In [ ]:
using DataFrames;
using CSV;


#file_name = "ep_gaussian4.csv"

# Define the header as an array of strings
row = ["t" "p_mom_var" "p_pos_var" "p_cc" "p_mom_mean" "p_pos_mean" "mom_var" "pos_var" "cc" "mom_mean" "pos_mean"]
header = DataFrame(row,vec(row))


t_vals = vec(t)
p_mom_var = vec(mom_var_plot.(t))
p_pos_var = vec(pos_var_plot.(t))
p_cc = vec(cross_cor_plot.(t))
p_mom_mean = vec(mom_mean.(t))
p_pos_mean = vec(mu.(t).+ mean_pos_cor.(t))
mom_var_vec = vec(c22.(t))
pos_var_vec = vec(c11.(t))
cc_vec = vec(c12.(t))
mom_mean_vec = vec(s2.(t))
pos_mean_vec = vec(s1.(t))


df = DataFrame([t_vals,p_mom_var,p_pos_var,p_cc,p_mom_mean,p_pos_mean,mom_var_vec,pos_var_vec,cc_vec,mom_mean_vec,pos_mean_vec],vec(row))



In [ ]:

# Write the header to a new CSV file
CSV.write(file_name, header;header =false)

CSV.write(file_name, df,append =true)


In [ ]:
df